# Detection of transmigration events with the *flowcatchR* package
Federico Marini, 3 February 2015 
-----------------------------

## Background

Platelets and leukocytes are known for playing essential roles in the development of inflammatory processes in blood vessels (http://www.nature.com/nm/journal/v17/n11/abs/nm.2515.html). The flow chamber system is an *in vitro* assay that allows for the investigation of biomolecular phenomena where the blood flow is simulated. 

The detection of candidate transmigration events for leukocytes occurs by mean of the detection of intensity changes of cells that, moving out of the focal plane of the microscope, are not anymore on the surface of the blood vessel.

## The analysis backbone

In [ ]:
source("http://bioconductor.org/biocLite.R")
biocLite("flowcatchR")
library("flowcatchR")

In [ ]:
# data loading
kerstinImages <- "/path/to/images/flowchambersystem/"
images <- list.files(kerstinImages,pattern = "*.png",full.names = T)

### Helper functions
Here I additionally define some helper functions, specific for this analysis.

In [ ]:
particleFeatures_alongTraj <- function(trajectoryset,
                                       particleset,
                                       trajID,
                                       ...)
{
  selectedTraj <- trajectoryset[[trajID]]$trajectory
  objectsToPick <- selectedTraj$frameobjectID
  framesToPick <- selectedTraj$frame
  extendedTrajInfo <- selectedTraj
  
  tmp <- lapply(1:nrow(selectedTraj),
         function(arg){
           # arg is the index pro point in the trajectory
           realFrame <- framesToPick[arg]
           frameObj <- objectsToPick[arg]
           objFeatures <- particleset[[ realFrame ]][frameObj,]
           objFeatures
         }
        )
  
  trajObjFeatures <- do.call(rbind.data.frame,tmp)
  trajFull <- data.frame(selectedTraj,trajObjFeatures)
  trajFull$meanIntensity <- trajFull$cell.0.s.area * trajFull$cell.a.b.mean
  return(trajFull)
}

# The analysis itself

In [ ]:
kjRaw <- read.Frames(images)
inspect.Frames(kjRaw,nframes=9,display.method="raster")
kjProc <- preprocess.Frames(kjRaw)
inspect.Frames(kjProc,nframes=9,display.method="raster")
kjParts <- particles(kjRaw,kjProc,channel = "all")
kjParts
kjTrajs <- trajectories(kjParts)
kjTrajs
plot2D.TrajectorySet(kjTrajs,kjRaw)

In [ ]:
trajID=4
traj4Feats <- particleFeatures_alongTraj(kjTrajs,kjParts,4)
framesToPick <- kjTrajs[[trajID]]$trajectory$frame
plot(framesToPick,traj4Feats$meanIntensity)
lines(spline(framesToPick,traj4Feats$meanIntensity),col = "steelblue")

par(mfrow=c(4,5))
for(i in 1:20)
{
  singleFullTraj <- particleFeatures_alongTraj(kjTrajs,kjParts,i)
  framesToPick <- kjTrajs[[i]]$trajectory$frame
  plot(framesToPick,singleFullTraj$meanIntensity,ylim=c(0,max(singleFullTraj$meanIntensity)))
  lines(spline(framesToPick,singleFullTraj$meanIntensity),col = "steelblue")
}


### Alternative workflow with extra normalization step
It is possible due to variation in the overall illumination that the image analysis is somehow encountering issues.
We try to correct here by normalizing the images on their intensity - this should give a better estimation of the intensity of the particle, fundamental to inspect an eventual decreasing of it as an indicator for a possible transmigration event.

In [ ]:
## slight modification by first normalizing the images in their intensity
kjNorm <- normalizeFrames(kjRaw)
kjProc_norm <- preprocess.Frames(kjNorm)
kjParts_norm <- particles(kjNorm,kjProc_norm,channel = "all")
kjTrajs_norm <- trajectories(kjParts_norm)
kjTrajs_norm
plot2D.TrajectorySet(kjTrajs_norm,kjNorm)


In [ ]:
## cfr on same trajectory: 4 for original data, 5 for normalized
par(mfrow=c(1,2))
trajID=4
traj4Feats <- particleFeatures_alongTraj(kjTrajs,kjParts,4)
framesToPick <- kjTrajs[[trajID]]$trajectory$frame
plot(framesToPick,traj4Feats$meanIntensity)
lines(spline(framesToPick,traj4Feats$meanIntensity),col = "steelblue")

trajID=5
traj5Feats_norm <- particleFeatures_alongTraj(kjTrajs_norm,kjParts_norm,5)
framesToPick <- kjTrajs_norm[[trajID]]$trajectory$frame
plot(framesToPick,traj5Feats_norm$meanIntensity)
lines(spline(framesToPick,traj5Feats_norm$meanIntensity),col = "steelblue")


In [ ]:
# true negative: 5 for original,6 for normalized
par(mfrow=c(1,2))
trajID=5
traj5Feats <- particleFeatures_alongTraj(kjTrajs,kjParts,5)
framesToPick <- kjTrajs[[trajID]]$trajectory$frame
plot(framesToPick,traj5Feats$meanIntensity)
lines(spline(framesToPick,traj5Feats$meanIntensity),col = "steelblue")

trajID=6
traj6Feats_norm <- particleFeatures_alongTraj(kjTrajs_norm,kjParts_norm,6)
framesToPick <- kjTrajs_norm[[trajID]]$trajectory$frame
plot(framesToPick,traj6Feats_norm$meanIntensity)
lines(spline(framesToPick,traj6Feats_norm$meanIntensity),col = "steelblue")


It is possible by taking a closer look to this *brightness profiles* that a reasonable and meaningful threshold is set, in order to detect location candidates for transmigration processes, where the cell fades out of the focal plane. This preliminary automated step serves as a way to reduce the user labour for inspecting the full datasets, by channeling the attention to particular locations in the field of view, and delivering quantitative measurements related to phenomena such as transmigration events which are normally evaluated just on a subjective level.

## Supplementary information

For more information on the method adapted for tracking cells, see Sbalzarini and Koumotsakos (2005).
For additional details regarding the functions of *flowcatchR*, please consult the documentation or write an email to (marinif@uni-mainz.de). 
For requesting help (e.g., for extending one of these analysis templates) or a particular functionality/submitting a pull request, please use https://github.com/federicomarini/flowcatchR and/or write an email to (marinif@uni-mainz.de).

## Acknowledgements
This package was developed at the Institute of Medical Biostatistics, Epidemiology and Informatics at the University Medical Center, Mainz (Germany), with the financial support provided by the TRP-A15 Translational Research Project grant. 

*flowcatchR* incorporates suggestions and feedback from the wet-lab biology units operating at the Center for Thrombosis and Hemostasis (CTH), in particular Sven Jäckel and Kerstin Jurk. Sven Jäckel also provided us with the sample acquisition which is available in the package vignette.

We would like to thank the members of the Biostatistics division for valuable discussions, and additionally Isabella Zwiener for contributing to the first ideas on the project.

## Session Information

In [ ]:
sessionInfo()